In [9]:
from getpass import getpass
from mysql.connector import connect, Error
import pandas as pd
import numpy as np
import datetime
import re


In [2]:
USER = 'root'

In [3]:
# проверка

try:
    with connect(
        host="localhost",
        user=USER,
        password=getpass("Пароль: "),
    ) as connection:
        print(connection)
except Error as e:
    print(e)

Пароль: ········


In [4]:
# создание базы

try:
    with connect(
        host="localhost",
        user=USER,
        password=getpass("Пароль: "),
    ) as connection:
        create_db_query = "CREATE DATABASE bank_statistics"
        with connection.cursor() as cursor:
            cursor.execute(create_db_query)
except Error as e:
    print(e)

Пароль: ········


In [18]:
# подключаемся сразу к нужной базе

try:
    with connect(
        host="localhost",
        user=USER,
        password=getpass("Пароль: "),
        database='bank_statistics'
        
    ) as connection:
        query = "SHOW TABLES"
        with connection.cursor() as cursor:
            cursor.execute(query)
            connection.commit()
except Error as e:
    print(e)

Пароль: ········
Unread result found


In [10]:
filials = pd.read_excel('Заубер Офисы.xlsx', sheet_name='адреса')

In [35]:
filials['Кассы'] = filials['Кассы'].fillna(0)
filials['Кассы'] = filials['Кассы'].astype(int)

In [28]:
# создадим таблицу filial_list (отделения)

try:
    with connect(
        host="localhost",
        user=USER,
        password=getpass("Пароль: "),
        database='bank_statistics'
        
    ) as connection:
        query = """CREATE TABLE filial_list
        (id INT PRIMARY KEY AUTO_INCREMENT,
        Наименование TEXT NOT NULL, 
        Талоны_наименование TEXT, 
        Банкиру_наименование TEXT,
       УМиР_Наименование TEXT, 
       Наименование_база TEXT, 
       Отделение_организации_база TEXT,
       Кассы INT, 
       Адрес TEXT, 
       Метро TEXT, 
       Город VARCHAR(50), 
       Текущий_статус TEXT)"""
        with connection.cursor() as cursor:
            cursor.execute(query)
            connection.commit()
except Error as e:
    print(e)

Пароль: ········


In [64]:
try:
    with connect(
        host="localhost",
        user=USER,
        password=getpass("Пароль: "),
        database='bank_statistics'
        
    ) as connection:
        alter_table_query = """
ALTER TABLE filial_list
MODIFY COLUMN Наименование VARCHAR(100)
"""
        with connection.cursor() as cursor:
            cursor.execute(query)
            connection.commit()
except Error as e:
    print(e)        

Пароль: ········
1822 (HY000): Failed to add the foreign key constraint. Missing index for constraint 'aggregate_indicators_ibfk_1' in the referenced table 'filial_list'


In [60]:
# проверим созданную таблицу

try:
    with connect(
        host="localhost",
        user=USER,
        password=getpass("Пароль: "),
        database='bank_statistics'
        
    ) as connection:
        query = "DESCRIBE filial_list"
        with connection.cursor() as cursor:
            cursor.execute(query)
            result = cursor.fetchall()
            for row in result:
                print(row)
except Error as e:
    print(e)

Пароль: ········
1146 (42S02): Table 'bank_statistics.aggregate_indicators' doesn't exist


In [43]:
filials = filials.fillna('')

In [44]:
filials_sql = [tuple(row) for index, row in filials.iterrows()]

In [72]:
# вставим значения из df


# проверим созданную таблицу

try:
    with connect(
        host="localhost",
        user=USER,
        password=getpass("Пароль: "),
        database='bank_statistics'
        
    ) as connection:
        query = """INSERT INTO filial_list
        (Наименование, 
        Талоны_наименование, 
        Банкиру_наименование,
       УМиР_Наименование, 
       Наименование_база, 
       Отделение_организации_база,
       Кассы, 
       Адрес, 
       Метро, 
       Город, 
       Текущий_статус)
       VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
        with connection.cursor() as cursor:
            cursor.executemany(query, filials_sql)
            connection.commit()
    
except Error as e:
    print(e)

Пароль: ········


In [50]:
# проверим записи

try:
    with connect(
        host="localhost",
        user=USER,
        password=getpass("Пароль: "),
        database='bank_statistics'
        
    ) as connection:
        query = "SELECT * FROM filial_list"
        with connection.cursor() as cursor:
            cursor.execute(query)
            result = cursor.fetchall()
            for row in result:
                print(row)
except Error as e:
    print(e)

Пароль: ········
(1, 'ГО', 'Головное отделение', 'Заубер Банк - м.Чкаловская, Большой пр-т ПС, д. 25/2', 'Головной офис АО «Заубер Банк»', 'Филиал ОАО "Заубер Банк" в Санкт-Петербурге', 'Головное отделение', 2, '197198, Санкт-Петербург, Большой проспект П.С., д.25/2', 'Чкаловская', 'Санкт-Петербург', 'Работает')
(2, 'Марата-ДО 1', 'ДО №1 ', 'Заубер Банк - м. Лиговский пр-т, ул.Марата, д. 56-58/29', 'Дополнительный офис №1 АО «Заубер Банк»', 'Дополнительный офис № 1  АО "Заубер Банк"', 'Дополнительный офис № 1', 2, '191119, Санкт-Петербург, ул. Марата 56-58/29', 'Лиговский', 'Санкт-Петербург', 'Работает')
(3, 'Комендантский-ОКВКУ 1', 'ОКВКУ №1 "Коменданский"', 'Заубер Банк - Комендантский пр-т, д. 16к1', 'Операционная касса вне кассового узла №1 «Комендантский»', 'ОКВКУ № 1 "Комендантский" АО "Заубер Банк"', 'ОКВКУ № 1 "Комендантский"', 2, '197372, Санкт-Петербург, Комендантский пр., д.16, корп.1', 'Комендантский', 'Санкт-Петербург', 'Работает')
(4, 'Лиговский-ОКВКУ 2', 'ОКВКУ №2 "На Ли

In [62]:
# проверка через pd.read_sql

with connect(
    host="localhost",
    user=USER,
    password=getpass("Пароль: "),
    database='bank_statistics'

) as connection:
    
    temp = pd.read_sql('SELECT * FROM filial_list', connection)

Пароль: ········


In [63]:
temp

id              Наименование          Талоны_наименование  \
0    1                        ГО           Головное отделение   
1    2               Марата-ДО 1                       ДО №1    
2    3     Комендантский-ОКВКУ 1      ОКВКУ №1 "Коменданский"   
3    4         Лиговский-ОКВКУ 2      ОКВКУ №2 "На Лиговском"   
4    5       Испытателей-ОКВКУ 3         ОКВКУ №3 Испытателей   
5    6     Международная-ОКВКУ 4     ОКВКУ №4 "Международная"   
6    7       Парк Победы-ОКВКУ 7       ОКВКУ №7 "Парк Победы"   
7    8           Озерки-ОКВКУ 12           ОКВКУ №12 "Озерки"   
8    9             Звездная-ДО 6             ДО №6 "Звездная"   
9   10         Наличная-ОКВКУ 13       ОКВКУ №13 "Приморская"   
10  11           Московский-ДО 2           ДО №2 "Московский"   
11  12           Рижская-ОКВКУ 5           ОКВКУ №5 "Рижская"   
12  13    Красные ворота-ОКВКУ 6    ОКВКУ №6 "Красные ворота"   
13  14        Павелецкая-ОКВКУ 8        ОКВКУ №8 "Павелецкая"   
14  15        Крылатское-ОКВКУ 9        ОКВКУ №9 "Крылатское"   
15  16             Жулебино-ДО 3             ДО №3 "Жулебино"   
16  17      Белорусская-ОКВКУ 10      ОКВКУ №10 "Белорусская"   
17  18               Пресня-ДО 4               ДО №4 "Пресня"   
18  19             Бибирево-ДО 5             ДО №5 "Бибирево"   
19  20  Новые Черемушки-ОКВКУ 11  ОКВКУ №11 "Новые Черемушки"   
20  21      Семеновская-ОКВКУ 14      ОКВКУ №14 "Семеновская"   
21  22       Спортивная-ОКВКУ 15       ОКВКУ №15 "Спортивная"   
22  23    Братиславская-ОКВКУ 16    ОКВКУ №16 "Братиславская"   
23  24      Теплый стан-ОКВКУ 17      ОКВКУ №17 "Тёплый Стан"   
24  25        Люблино - ОКВКУ 18          ОКВКУ №18 "Люблино"   
25  26    Заубер Сити - ОКВКУ 19       ОКВКУ №19 "ЗауберСити"   
26  27     Войковская - ОКВКУ 20       ОКВКУ №20 "Войковская"   
27  28      Кузьминки - ОКВКУ 21        ОКВКУ №21 "Кузьминки"   
28  29   Полежаевская - ОКВКУ 22     ОКВКУ №22 "Полежаевская"   
29  30      Чертаново - ОКВКУ 23        ОКВКУ №23 "Чертаново"   
30  31       Тульская - ОКВКУ 24        ОКВКУ № 24 "Тульская"   
31  32      Филиал в г. Черкесск                                
32  33             ККО Краснодар                                
33  34                 ОКВКУ № 3                                
34  35                 ОКВКУ № 5                                
35  36                 ОКВКУ № 6                                
36  37                 ОКВКУ № 3                                

                                 Банкиру_наименование  \
0   Заубер Банк - м.Чкаловская, Большой пр-т ПС, д...   
1   Заубер Банк - м. Лиговский пр-т, ул.Марата, д....   
2           Заубер Банк - Комендантский пр-т, д. 16к1   
3   Заубер Банк - м. Пл.Восстания, Лиговский пр., ...   
4   Заубер Банк - м.Комендантский пр-т, пр-т Испыт...   
5     Заубер Банк - м. Международная, ул. Б.Куна, д.8   
6   Заубер Банк - м. Парк Победы, Московский пр-т,...   
7       Заубер Банк - м. Озерки, пр-т Энгельса, 124к1   
8       Заубер Банк - м. Звездная, ул.Звездная, д.5к1   
9     Заубер Банк - м. Наличная, ул.Наличная, д. 40к1   
10  Заубер Банк - м.Новокузнецкая, ул. Садовническ...   
11             Заубер Банк - м.Рижская, пр-т Мира, 79   
12  Заубер Банк - м.Красные Ворота, Садовая-Черног...   
13     Заубер Банк - м.Павелецкая, Зацепский Вал, д.5   
14    Заубер Банк - м.Крылатское, Осенний б-р, д.10к1   
15  Заубер Банк - м.Жулебино, ул. Генерала Кузнецо...   
16  Заубер Банк - м.Белорусская, Ленинградский пр-...   
17  Заубер Банк - м. Улица 1905 года, Пресненский ...   
18  Заубер Банк - м.Бибирево, Бибиревская ул., д.10/1   
19  Заубер Банк - м.Новые Черемушки, ул. Гарибальд...   
20   Заубер Банк - м.Семеновская, Семёновская пл., д7   
21    Заубер Банк - м.Спортивная, ул. Усачёва, д.29к1   
22  Заубер Банк - м.Братиславская, Братиславская у...   
23  Заубер Банк - м.Теплый Стан, Новоясеневский пр...   
24       Заубер Банк - м.Люблино, Совхозная ул., д.39   
25  Заубер Банк - м. Международная, ул. Тестовская.

## создадим таблицу для агрегированной банковской статистики aggregate_indicators

In [73]:
# создадим таблицу filial_list (отделения)

try:
    with connect(
        host="localhost",
        user=USER,
        password=getpass("Пароль: "),
        database='bank_statistics'
        
    ) as connection:
        query = """CREATE TABLE aggregate_indicators
        (id INT PRIMARY KEY AUTO_INCREMENT,
        Филиал VARCHAR(100), 
        Дата DATE, 
        Доход DOUBLE,
       Приход_USD DOUBLE, 
       Расход_USD DOUBLE, 
       Приход_EUR DOUBLE,
       Расход_EUR DOUBLE,
       Кол_во_операций INT, 
       Остаток_в_кассе_кубли DOUBLE
       )"""
        with connection.cursor() as cursor:
            cursor.execute(query)
            connection.commit()
except Error as e:
    print(e)

Пароль: ········


In [68]:
# проверим созданную таблицу

try:
    with connect(
        host="localhost",
        user=USER,
        password=getpass("Пароль: "),
        database='bank_statistics'
        
    ) as connection:
        query = "DESCRIBE aggregate_indicators"
        with connection.cursor() as cursor:
            cursor.execute(query)
            result = cursor.fetchall()
            for row in result:
                print(row)
except Error as e:
    print(e)

Пароль: ········
('id', b'int', 'NO', 'PRI', None, 'auto_increment')
('Филиал', b'varchar(100)', 'YES', '', None, '')
('Дата', b'text', 'YES', '', None, '')
('Доход', b'double', 'YES', '', None, '')
('Приход_USD', b'double', 'YES', '', None, '')
('Расход_USD', b'double', 'YES', '', None, '')
('Расход_EUR', b'double', 'YES', '', None, '')
('Кол_во_операций', b'int', 'YES', '', None, '')
('Остаток_в_кассе_кубли', b'double', 'YES', '', None, '')


In [69]:
try:
    with connect(
        host="localhost",
        user=USER,
        password=getpass("Пароль: "),
        database='bank_statistics'
        
    ) as connection:
        alter_table_query = """
ALTER TABLE aggregate_indicators
MODIFY COLUMN Дата DATE
"""
        with connection.cursor() as cursor:
            cursor.execute(query)
            connection.commit()
except Error as e:
    print(e)     

Пароль: ········
1045 (28000): Access denied for user 'root'@'localhost' (using password: YES)
